In [16]:
import pandas as pd
import numpy as np
from glmnet import LogitNet
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.datasets import load_svmlight_files
from sklearn.metrics import accuracy_score
import xgboost as xgb
import csv
from xgboost import XGBRegressor
from sklearn import preprocessing
import seaborn as sns
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

### Computer system spec
MacBook Air (13-inch, Early 2014)

Processor 1.4 GHz Intel Core i5

Memory 4 GB 1600 MHz DDR3

### Load the dataset Ames_data.csv

In [17]:
# read csv file
df = pd.read_csv('/Users/glen/Desktop/Statistical learning/Project/Ames_data.csv')
# df.head(5)

In [18]:
# Descriptive statistics for each column
# df.describe()

In [19]:
# df shape
testnum = int(round(df.shape[0]*0.3 , 0))
print(df.shape)
print("Test number:", testnum)

#make sure PID is unique
print("unique PID number",pd.unique(df['PID']).size)

(2930, 83)
Test number: 879
unique PID number 2930


### Data preprocessing
Because I try to use one-hot encoding later. In order to decrease the number of categories. First I delete the column if its categories exceed 15. Training dataset and test dataset would get different number of columns while doing one-hot encoding, so I need to align the X_train and X_test together to get the same number of column. Then I use imputer to replace the Nan value by the mean value. 

In [36]:
#data preprocessing

#delete column 'Street', 'Longtitude', 'Latitude'...
df = df.drop(columns = ['Street', 'Utilities', 'Land_Slope', 'Condition_2', 'Roof_Matl', 'Heating', 'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Three_season_porch', 'Pool_Area', 'Misc_Val', 'Longitude', 'Latitude'])

# #Choose column
# choose_column = [col for col in X_train.columns]

# X_train = X_train[choose_column]
# X_test = X_test[choose_column]

print("dataset contains {0} rows and {1} columns".
      format(df.shape[0], df.shape[1]))

dataset contains 2930 rows and 69 columns


In [37]:
#implement one hot encoding to get the result
df = pd.get_dummies(df)

print("dataset after one hot contains {0} rows and {1} columns".
      format(X_test.shape[0], X_test.shape[1]))

dataset after one hot contains 879 rows and 303 columns


### Split the data into 70% training and 30% testing dataset

In [38]:
# Split data in input and output
original_y = df['Sale_Price']
original_X = df.drop(columns = ['Sale_Price'])

#Split data in train and test set
X_train, X_test, y_train, y_test = train_test_split(
    original_X, original_y, test_size=0.3,random_state=12)

print("Train dataset contains {0} rows and {1} columns".
      format(X_train.shape[0], X_train.shape[1]))
print("Test dataset contains {0} rows and {1} columns".
      format(X_test.shape[0], X_test.shape[1]))

#Save the train.csv and test.csv file

X_train.to_csv('train.csv', index = False)
X_test.join(y_test).to_csv('test.csv', index = False)

# pd.merge(X_test,y_test,left_on='index',right_on='index')


Train dataset contains 2051 rows and 307 columns
Test dataset contains 879 rows and 307 columns


In [39]:
#Use Imputer to deal with missing value or nan
#train_X and test_X are numpy array
my_imputer = Imputer(missing_values='NaN', strategy='mean')
train_X = my_imputer.fit_transform(X_train)
test_X = my_imputer.transform(X_test)
print("Train dataset contains {0} rows and {1} columns".
      format(train_X.shape[0], train_X.shape[1]))
print("Test dataset contains {0} rows and {1} columns".
      format(test_X.shape[0], test_X.shape[1]))

Train dataset contains 2051 rows and 307 columns
Test dataset contains 879 rows and 307 columns


### Save X_train and y_train in csv files. Then I use glmnet in R to predict the best lambda.  

In [40]:
# save X_train & X_test as csv file which are used to compute best lambda
# best lambda = 763.7335
X_train.fillna(0)
X_train.to_csv('X_train.csv', index = False)
y_train.to_csv('y_train.csv', index = False)

In [ ]:
train_X

### Coordinate Descent lasso algorithm
Use coordinate descent to calculate the result. Unfortunately, I can't get a decent result from this model. My smallest RMSE value is 0.48

In [44]:
# implement Lasso using Coordinate Descent
def one_step_lasso(r, x, lam):
    # x: nx1 matrix
    # r: nx1 matrix
    # Use the soft-thresholding result lasso_j = sgn(βLS_j)(|βLSj|−γ)+
    # return beta_j
    xx = np.sum(np.square(x))
    xr = np.sum(x*r)
    b = (abs(xr) -lam/2)/xx
    b = np.sign(xr)*(b if b>0 else 0) 
    return b

def mylasso(X, y, lam, n_iter, standardize = False):
    # X: n-by-p design matrix without the intercept
    # y: n-by-1 response vector
    # p: p-by-1 vector
    # lam: lambda value
    # n.iter: number of iterations
    # standardize: if True, center and scale X and y. 
    
    #p is the number of features (columns)
    p = X.shape[1]
    
    # n is the number of rows
    n = X.shape[0]
    
    # YOUR CODE
    # If standardize  = TRUE, center and scale X and Y 
    # record the corresponding means and sd
    # scaler = StandardScaler()
    # X = scaler.fit_transform(X)
    # y = scaler.fit_transform(y)
    mean_y = np.mean(y)
    mean_X = np.mean(X,axis = 0)
    sd_y = np.std(y)
    sd_X = np.std(X,axis = 0)
    
    if standardize == True:
        mean_y = np.mean(y)
        mean_X = np.mean(X,axis = 0)
        sd_y = np.std(y)
        sd_X = np.std(X,axis = 0)
        X = (X - np.mean(X,axis = 0))/np.std(X,axis = 0)
        y = (y - np.mean(y))/np.std(y)

    # Initial values for residual and coefficient vector b
    # b: p-by-1 vector, without intercept
        
    b = np.zeros(p)
    r = y

    
    for step in range(n_iter):
        for j in range(p):
        # YOUR CODE 
        # 1) Update the residual vector to be the one
        # in blue on p37 of [lec_W3_VariableSelection.pdf]. 
        # r <-- current residual + X[, j] * b[j]
        # r is n-by-1 vector

            r = r + X[:,j]*b[j]
        # 2) Apply one_step_lasso to update beta_j
        # b[j] = one_step_lasso(r, X[, j], lam)
            b[j] = one_step_lasso(r, X[:,j], lam)

#         # 3) Update the current residual vector
#         # r <-- r - X[, j] * b[j]
            r = r - X[:,j]*b[j]
    # YOUR CODE: scale back b and add intercept b0
    # For b0, check p13 of [lec_W3_VariableSelection.pdf]. 

#     b = (sd_y/sd_X)*b
    b_intercept = mean_y - np.dot(mean_X.T,b)
    return (b_intercept,b)

In [46]:
lam = 0.013
X = train_X
y = y_train
n_iter = 100
standardize = True
b_intercept = mylasso(X, y, lam, n_iter)[0]
b = mylasso(X, y, lam, n_iter)[1].reshape(307,1)
# np.unique(b)

/Users/glen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


In [47]:
pre_test_y = np.dot(test_X,b) + b_intercept
ans = round(math.sqrt(np.mean(np.square(
    np.log(abs(pre_test_y)) - np.array(np.log(y_test))))), 4)
print("Root-Mean-Squared-Error (RMSE) for Coordinate decent lasso model is {0}".
      format(ans))

Root-Mean-Squared-Error (RMSE) for Coordinate decent lasso model is 0.9146


In [14]:
# export the result as mysubmission3.txt
PID = np.array(X_test['PID'].values).reshape(len(X_test),1)
Sale_Price = pre_test_y.reshape(len(pre_test_y),1)
ans = np.concatenate((PID,Sale_Price),axis=1)

# save the data with mixted datatype, %d saves PID as integer and 
# %10.5f can save the sale_price as double 
np.savetxt("mysubmission3", ans, delimiter=',', 
           header="PID,Sale_Price", fmt='%d %10.5f', comments='')